In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


# Lecture 02: Single Agent Stock Analysis with Structured Output

This notebook demonstrates a single AI agent that uses web search to analyze stocks and generate comprehensive reports.

## Key Concepts:
- **Structured Output**: The agent uses Pydantic models (`StockReport`) to ensure responses follow a specific, validated schema
- **Type Safety**: By specifying `output_type=StockReport`, the agent is guaranteed to return structured data with all required fields
- **Web Search Integration**: The agent can search the web for real-time stock information to populate the report

The `StockReport` model defines fields like `symbol`, `company_name`, `current_price`, `recommendation`, and more, ensuring consistent and predictable output from the AI agent.


In [ ]:
# Setup: Import necessary modules
from pydantic_ai import Agent
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging, get_current_date

# Initialize logging
setup_logging()

In [3]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="stock-analysis-lecture02",
    start_message="🚀 Lecture 02 - Stock Analysis Notebook Started",
)

In [4]:
# Define the StockReport model for structured output
from typing import List, Optional
from pydantic import BaseModel, Field


class StockReport(BaseModel):
    """Comprehensive stock analysis report."""

    symbol: str = Field(description="Stock symbol (e.g., AAPL, TSLA)")
    company_name: str = Field(description="Full company name")
    current_price: Optional[float] = Field(
        description="Current stock price", default=None
    )
    price_change: Optional[float] = Field(
        description="Price change from previous close", default=None
    )
    price_change_percent: Optional[float] = Field(
        description="Price change percentage", default=None
    )

    # Analysis sections
    executive_summary: str = Field(description="Brief executive summary of the stock")
    recent_news: List[str] = Field(
        description="Key recent news items affecting the stock", default_factory=list
    )
    financial_highlights: List[str] = Field(
        description="Key financial metrics and highlights", default_factory=list
    )
    market_sentiment: str = Field(description="Overall market sentiment analysis")
    risk_factors: List[str] = Field(
        description="Key risk factors to consider", default_factory=list
    )
    recommendation: str = Field(description="Investment recommendation with reasoning")

    # Metadata
    analysis_date: str = Field(description="Date when analysis was performed")
    data_sources: List[str] = Field(
        description="Sources used for the analysis", default_factory=list
    )


print("✅ StockReport model defined")

✅ StockReport model defined


In [5]:
# Create the stock analysis agent with structured output
stock_analysis_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool],
    output_type=StockReport,  # Returns structured StockReport
    system_prompt="""
You are an expert financial analyst specializing in stock analysis and market research.

Your task is to create comprehensive stock reports by gathering and analyzing current information.

When analyzing a stock, you should:
1. **Company Overview**: Get basic company information and recent developments
2. **Financial Performance**: Look for recent earnings, revenue trends, and key metrics
3. **Market Position**: Research competitive landscape and market share
4. **Recent News**: Find recent news that could impact the stock price
5. **Market Sentiment**: Gauge overall market sentiment toward the stock
6. **Risk Assessment**: Identify potential risks and challenges

Provide a structured analysis with specific data points when available.
""".strip(),
)

In [ ]:
# Analyze a stock
stock_symbol = "AAPL"
print(f"Analyzing {stock_symbol}...\n")

prompt = f"Please analyze the stock {stock_symbol}. The analysis date should be: {get_current_date()}"
result = await stock_analysis_agent.run(prompt)
report = result.output

# Display the report
print(f"{'='*80}")
print(f"Stock Report for {report.symbol}")
print(f"Company: {report.company_name}")
print(f"Analysis Date: {report.analysis_date}")
if report.current_price:
    print(f"Current Price: ${report.current_price:.2f}")
    if report.price_change and report.price_change_percent:
        print(
            f"Change: ${report.price_change:.2f} ({report.price_change_percent:.2f}%)"
        )

print(f"\n{'='*80}")
print("EXECUTIVE SUMMARY")
print(f"{'='*80}")
print(report.executive_summary)

if report.recent_news:
    print(f"\n{'='*80}")
    print("RECENT NEWS")
    print(f"{'='*80}")
    for i, news in enumerate(report.recent_news, 1):
        print(f"{i}. {news}")

if report.financial_highlights:
    print(f"\n{'='*80}")
    print("FINANCIAL HIGHLIGHTS")
    print(f"{'='*80}")
    for highlight in report.financial_highlights:
        print(f"• {highlight}")

print(f"\n{'='*80}")
print("MARKET SENTIMENT")
print(f"{'='*80}")
print(report.market_sentiment)

if report.risk_factors:
    print(f"\n{'='*80}")
    print("RISK FACTORS")
    print(f"{'='*80}")
    for i, risk in enumerate(report.risk_factors, 1):
        print(f"{i}. {risk}")

print(f"\n{'='*80}")
print("RECOMMENDATION")
print(f"{'='*80}")
print(report.recommendation)
print(f"{'='*80}")

Analyzing AAPL...

Stock Report for AAPL
Company: Apple Inc.
Analysis Date: 2025-11-30
Current Price: $272.50
Change: $2.50 (0.92%)

EXECUTIVE SUMMARY
Apple Inc. remains a dominant force in the global technology sector, reporting record quarterly and annual revenue in fiscal 2025 and continuing to innovate with products and services, notably the iPhone 17 and new AI-powered software features. The company’s services business and hardware launches have maintained strong momentum, even as Apple faces heightened competition and regulatory pressures worldwide. The stock has performed strongly, reflecting bullish technical and fundamental trends.

RECENT NEWS
1. Apple reports record Q4 2025 earnings: $102.47B revenue, $27.47B net income, EPS $1.85 (9to5Mac, Apple)
2. Apple plans largest-ever iPhone upgrade series in 2026, including foldable models, enhanced AI (PYMNTS, 9to5Mac)
3. Apple faces $38B antitrust fine in India but sees iPhone sales surge from Singles’ Day (CNN)
4. Apple announces 

In [ ]:
# Try another stock
stock_symbol = "TSLA"
print(f"Analyzing {stock_symbol}...\n")

prompt = f"Please analyze the stock {stock_symbol}. The analysis date should be: {get_current_date()}"
result = await stock_analysis_agent.run(prompt)
report = result.output

# Display a shorter version of the report
print(f"{'='*80}")
print(f"Stock Report for {report.symbol}")
print(f"Company: {report.company_name}")
if report.current_price:
    print(f"Current Price: ${report.current_price:.2f}")

print(f"\n{'='*80}")
print("EXECUTIVE SUMMARY")
print(f"{'='*80}")
print(report.executive_summary)

print(f"\n{'='*80}")
print("RECOMMENDATION")
print(f"{'='*80}")
print(report.recommendation)
print(f"{'='*80}")